In [10]:
from __importer import current_file_dir
from app.agents.student_agent import StudentAgent

%reload_ext autoreload
%autoreload 2

In [11]:
my_agent = StudentAgent()
my_agent.get_tasks()

[{'id': 1, 'title': 'Disease', 'description': 'Disease description?'},
 {'id': 2, 'title': 'Prognosis', 'description': 'Prognosis description?'}]

In [16]:
my_agent.start_session(1)

{'session_id': 48813,
 'history': [{'role': 'user',
   'content': 'Hi, doctor. I’ve been having trouble breathing for the past few days. It started about four days ago and seems to be getting worse. I feel short of breath, especially when I move around or try to take a deep breath. I also have this dry cough that won’t go away. There’s no real chest pain, but I do feel a bit uncomfortable in my chest when I breathe in deeply. I’ve felt more tired than usual, and I’ve had some mild fevers at home. I just don’t feel like myself.'}],
 'score': None,
 'is_end': False}

In [ ]:
my